In [ ]:
import tkinter as tk
from tkinter import messagebox
import lib_scen_intersection as lsi
import lib_map_draw as dr
import lib_spawn as spb
import lib_traffic as trf
import lib_sensors as sen
import carla
from importlib import reload
reload(lsi) #per refreshare modifiche import
reload(dr)
reload(spb)
reload(trf)
reload(sen)

class TrafficSimulatorGUI:
    def __init__(self, root, spacing=10):
        self.root = root
        self.root.title("Junction Scenarios")
        self.spacing = spacing
        self.world, self.crossings = self.connect_sim();
        
        #strutture dati per incroci
        self.spawns = None    #lista di coppie ( spawns, wpdestinazione ) disponibili all'incrocio selezionato
        self.gr_pairs = None  #lista di adiancenza di coppie di wp dove la key è il wp di intersezione
        self.crossing_index = 0
        self.spawn_index = 0

        self.ego_vehicle = None #(vehicle, wp)
        self.npc_vehicles = None
        
        # Bottone per resettare il simulatore
        tk.Button(root, text="Reset Simulator", command=self.reset_simulator).grid(row=0, column=0, columnspan=4, pady=self.spacing)

        # Incrocio - bottoni
        self.create_navigation_section("Junction", self.crossings,
                                       "Prev Junction", "Next Junction", 1)

        # Spawn - bottoni
        self.create_navigation_section("Spawn", self.spawns,
                                       "Prev Spawn", "Next Spawn", 2)

        # Bottone per "Spawn veicolo" e checkbox
        tk.Button(root, text="Spawn vehicle", command=self.spawn_vehicle).grid(row=3, column=0, pady=self.spacing)
        self.ignore_traffic_lights = tk.BooleanVar(value=False)  # Variabile per lo stato della checkbox
        tk.Checkbutton(root, text="Skip Traffic Lights", variable=self.ignore_traffic_lights).grid(row=3, column=1, padx=self.spacing)
        
        # Textbox e bottone per "Genera traffico"
        tk.Label(root, text="BOT number:").grid(row=4, column=0, padx=self.spacing, pady=self.spacing)
        self.traffic_entry = tk.Entry(root, width=8)  
        self.traffic_entry.grid(row=4, column=1, padx=self.spacing, pady=self.spacing)
        tk.Button(root, text="Fill Junction", command=self.fill_junction).grid(row=4, column=2, padx=self.spacing, pady=self.spacing)

        # Menù a tendina per profilo guida spwan
        tk.Label(root, text="Ego Agent Profile:").grid(row=5, column=0, padx=self.spacing, pady=self.spacing)
        self.spawn_profile = tk.StringVar(value="Select")
        spawn_menu = tk.OptionMenu(root, self.spawn_profile, "Cautious", "Normal", "Aggressive",
                                   command=self.display_spawn_profile)
        spawn_menu.grid(row=5, column=1, padx=self.spacing, pady=self.spacing)

        # Menù a tendina per profilo guida traffico
        tk.Label(root, text="Traffic Agent Profile:").grid(row=6, column=0, padx=self.spacing, pady=self.spacing)
        self.traffic_profile = tk.StringVar(value="Select")
        traffic_menu = tk.OptionMenu(self.root, self.traffic_profile, "Cautious", "Normal", "Aggressive",
                                     command=self.display_traffic_profile)
        traffic_menu.grid(row=6, column=1, padx=self.spacing, pady=self.spacing)
        
        # BOTTONE CHE AVVIA SIMULAZIONE
        tk.Button(root, text="Start simulation", command=self.start_simulation).grid(row=10, column=0, columnspan=4, pady=self.spacing)

        # BOTTONE CHE STAMPA I PERCORSI CHE SI INCROCIANO IN UNO STESSO WP DI DESINAZIONE
        tk.Button(root, text="Draw common routes", command=self.draw_common_route).grid(row=3, column=2, padx=self.spacing)
        # BOTTONE CHE SPAWNA VEICOLI SUI PERCORSI CHE SI INCROCIANO IN UNO STESSO WP DI DESINAZIONE
        tk.Button(root, text="Spawn on common routes", command=self.spawn_common_route).grid(row=3, column=3, padx=self.spacing)

    #FUNZIONE CHE GESTISCE I BOTTONI DI SCELTA INCROCIO E SPAWN POINTS
    def create_navigation_section(self, label, array, left_text, right_text, row):

        draw_time = 60
        
        tk.Label(self.root, text=f"{label} attuale:").grid(row=row, column=0, padx=self.spacing, pady=self.spacing)
        
        #ETICHETTE ALL'AVVIO
        if label == "Junction":
           display_label = tk.Label(self.root, text=f"{self.crossing_index + 1}/{len(self.crossings)}")
           
            # dato l'incrocio, recupero la lista di coppie di wp che lo circondano indicizzate da self.crossing_index
           self.gr_pairs = lsi.group_pairs_by_second_waypoint(self.crossings[self.crossing_index][1].get_waypoints(carla.LaneType.Driving) )
           self.spawns = spb.list_all_behind_wp_couple(self.world, self.gr_pairs, 5)
           
           dr.focus_above(self.world, self.spawns[0][0].location, 60, -90)
           dr.draw_spawn(self.world, self.spawns[0][0], draw_time, f"SP_{self.crossing_index + 1}_{self.spawn_index + 1}", 0, 255, 0)
            
        elif label == "Spawn":
            display_label = tk.Label(self.root, text=f"{self.spawn_index + 1}/{len(self.spawns)}")
            
            dr.focus_above(self.world, self.spawns[self.spawn_index][0].location, 60, -90)
            dr.draw_spawn(self.world, self.spawns[self.spawn_index][0], draw_time, f"SP_{self.crossing_index + 1}_{self.spawn_index + 1}", 0, 255, 0)
            
        display_label.grid(row=row, column=1, padx=self.spacing, pady=self.spacing)
        
        #FUNZIONE ATTIVATA ALLA PRESSIONE DEI BOTTONI DI INCROCI E SPWANS
        def update_label(direction):
            nonlocal display_label
            if label == "Junction":
                dr.pair_counter = 0
                self.crossing_index = (self.crossing_index + direction) % len(self.crossings)
                display_label.config(text=f"{self.crossing_index  + 1 }/{len(self.crossings)}")#self.crossings[self.crossing_index][0]) #hunction contiene coppie id, coordinate
                
                # dato l'incrocio, recupero la lista di coppie di wp che lo circondano indicizzate da self.crossing_index
                self.gr_pairs = lsi.group_pairs_by_second_waypoint(self.crossings[self.crossing_index][1].get_waypoints(carla.LaneType.Driving) )
                self.spawns = spb.list_all_behind_wp_couple(self.world, self.gr_pairs, 5)
                
                dr.focus_above(self.world, self.spawns[0][0].location, 60, -90)
                dr.draw_spawn(self.world, self.spawns[0][0], draw_time, f"SP_{self.crossing_index + 1}_{self.spawn_index + 1}", 0, 255, 0)
                self.spawn_index = 0
                #display_label.config(text=f"{self.spawn_index + 1}/{len(self.spawns)}")
                
            elif label == "Spawn":
                self.spawn_index = (self.spawn_index + direction) % len(self.spawns)
                display_label.config(text=f"{self.spawn_index + 1}/{len(self.spawns)}")
                
                dr.focus_above(self.world, self.spawns[self.spawn_index][0].location, 60, -90)
                dr.draw_spawn(self.world, self.spawns[self.spawn_index][0], draw_time, f"SP_{self.crossing_index + 1}_{self.spawn_index + 1}", 0, 255, 0)

        tk.Button(self.root, text=left_text, command=lambda: update_label(-1)).grid(row=row, column=2, padx=self.spacing, pady=self.spacing)
        tk.Button(self.root, text=right_text, command=lambda: update_label(1)).grid(row=row, column=3, padx=self.spacing, pady=self.spacing)
    
    #ELIMINA VEICOLI E SENSORI DAL SIMULATORE
    def reset_simulator(self):
        # Resetta gli indici
        self.crossing_index = 0
        self.spawn_index = 0

        self.ego_vehicle = None
        self.npc_vehicles = None

        # Resetta i campi di input
        self.traffic_entry.delete(0, tk.END)

        # Resetta i profili selezionati
        self.spawn_profile.set("Select")
        self.traffic_profile.set("Select")

        # Resetta lo stato della checkbox
        self.ignore_traffic_lights.set(False)

        #cancella tutti gli attori dal simulatore
        lsi.reset_sim(self.world)

        # Messaggio di conferma
        messagebox.showinfo("Reset Copmpleted", "All actors and sensors destroyed.")

    #BOTTONE SPAWN EGO VEHICLE
    def spawn_vehicle(self):
        traffic_lights_status = "Traffic Light ignored" if self.ignore_traffic_lights.get() else "Traffic Light observed"
        #l'ego vehicle diventa quello spwnato alla pressione del bottone di spawn
        self.ego_vehicle = spb.spawn_vehicle(self.world, self.spawns[self.spawn_index])
        print(f"stampo ego {self.ego_vehicle}")
        #messagebox.showinfo("Spawn Vehicle", f"Hello, world!\n{traffic_lights_status}")

    #SPAWNA VEICOLI IN TUTTO L'INCROCIO
    def fill_junction(self):
        self.npc_vehicles = spb.spawn_traffic(self.world, self.spawns) 

    def display_spawn_profile(self, selection):
        print("Spawn agent profile", f"Profilo selezionato per Spwan: {selection}")

    def display_traffic_profile(self, selection):
        print("Traffic profile", f"Profilo selezionato per Traffico: {selection}")

    def start_simulation(self):
        # Raccoglie i parametri selezionati
        crossing = self.crossings[self.crossing_index]
        spawn = self.spawns[self.spawn_index]
        traffic_value = self.traffic_entry.get()
        spawn_profile = self.spawn_profile.get().lower()
        traffic_profile = self.traffic_profile.get().lower()
        traffic_lights_status = "Semafori ignorati" if self.ignore_traffic_lights.get() else "Semafori osservati"

        # Controlla siano settati i profili pena errore
        try:
            profiles = ["cautious", "normal", "aggressive"]
            msg = ""
            
            if spawn_profile not in profiles:
                msg = "Choose a driving profile for Ego vehicle!!"
                raise ValueError
            if self.npc_vehicles is not None and traffic_profile not in profiles:
                msg = "Choose a driving profile for spawned Traffic!!"
                raise ValueError
            if self.ego_vehicle is None:
                msg = "No Ego vehicle spawned!!"
                raise ValueError
                
        except ValueError:
            messagebox.showinfo("ERROR: ", msg)
            return

        summary = (f"Simulazione avviata con i seguenti parametri:\n"
                   f"- Incrocio selezionato: {crossing}\n"
                   f"- Spawn selezionato: {spawn}\n"
                   f"- Veicoli da generare: {traffic_value}\n"
                   f"- Profilo guida spwan: {spawn_profile}\n"
                   f"- Profilo guida traffico: {traffic_profile}\n"
                   f"- Stato semafori: {traffic_lights_status}")
        messagebox.showinfo("Avvio Simulazione", summary)

        #AVVIA EGO VEHICLE spawnato a self.spawns[self.spawn_index] self.ego_vehicle
        trf.start_vehicle(self.world, self.ego_vehicle, 'EGO_VEHICLE', visible = 2, behaviour = spawn_profile)
        #sensori per veicolo
        sen.attach_sensors(self.world, self.ego_vehicle[0])
        #AVVIA TRAFFICO SE SELEZIONATO
        if self.npc_vehicles is not None:
           trf.start_vehicle_list(self.world, self.npc_vehicles, 'NPC', visible = 1, behaviour = traffic_profile)

    #SIM CONNECTION 
    def connect_sim(self):
        client = carla.Client('localhost', 2000)
        client.set_timeout(5.0)
        world = client.get_world()
        bp_lib = world.get_blueprint_library() 
        map = world.get_map()
        
        # lista di wp alternati da dist di 2 metri
        waypoints = map.generate_waypoints(2.0)
        
        # array di incroci contenente coppie (id, oggetto_incrocio)
        unique_junctions = []
        
        # lettura dei wp eventualmente presenti ad un' intersezione
        #N.B.: più waypoint possono appartenere a uno stesso incrocio
        for waypoint in waypoints:
            if waypoint.is_junction:
                junction = waypoint.get_junction()
                if not(any(u[0] == junction.id for u in unique_junctions)):
                    unique_junctions.append( (junction.id, junction) )
        
        # Stampa il numero di incroci (9 distinti in town 10)
        print(f"Numero di incroci trovati: {len(unique_junctions)}\n") 
        
        return (world, unique_junctions)
    
    #funzione che dato una lista e un wp, disegna tutte le route che partono da un sp e vanno in quel wp
    def draw_common_route(self):
        wp2 = self.spawns[self.spawn_index][1]
        key_wp2 = (round(wp2.transform.location.x, 3), round(wp2.transform.location.y, 3))
        dr.draw_intersection(self.world, self.gr_pairs[key_wp2])

    #funzione spawna veicoli su tutte le route accomunate da wp2 come destinazione
    def spawn_common_route(self):
        wp2 = self.spawns[self.spawn_index][1]
        key_wp2 = (round(wp2.transform.location.x, 3), round(wp2.transform.location.y, 3))
        current_sp = self.spawns[self.spawn_index][0]
        
        for sp, wp_dest in self.spawns:
            wp_d = (round(wp_dest.transform.location.x, 3), round(wp_dest.transform.location.y, 3))
            if key_wp2 == wp_d:
               spb.spawn_vehicle(self.world, (sp, wp_dest))


# Avvio dell'interfaccia
root = tk.Tk()
app = TrafficSimulatorGUI(root, spacing=7)  # Personalizza la spaziatura qui
root.mainloop()


Numero di incroci trovati: 9

stampo ego (<carla.libcarla.Vehicle object at 0x000001BC7E486340>, <carla.libcarla.Waypoint object at 0x000001BC7E487680>)
stampo ego (<carla.libcarla.Vehicle object at 0x000001BC7E4862D0>, <carla.libcarla.Waypoint object at 0x000001BC7E485A80>)
Spawn agent profile Profilo selezionato per Spwan: Normal
in start (<carla.libcarla.Vehicle object at 0x000001BC7E4862D0>, <carla.libcarla.Waypoint object at 0x000001BC7E485A80>)
Il veicolo Actor(id=56, type=vehicle.lincoln.mkz_2020) ha raggiunto la sua destinazione (Stop forzato=False)
Destinazione raggiunta, Fine misurazione
